In [2]:
import pandas as pd
import numpy as np

### Length features learning
First, we'll load our dataframe and convert it into the rigth format

In [3]:
labels = pd.read_csv("Results/Machine_Learning/sample_map_ML.csv")
features  = pd.read_csv("Results/Machine_Learning/fragment_features_CNA_ML.csv")
# convert to long
features = pd.pivot(data = features, index = "sample", columns = "length", values = "count")

In [4]:
cancer_list = ["A", "B", "C", "Lung", "Breast"]

def binary(df):
    df["cancer"] = np.where(df["diagnosis"].isin(cancer_list), 1, 0)
    
    return df

# apply function and reshape df
labels = binary(labels)
labels.drop(columns = ["diagnosis"], inplace = True)
labels.set_index("sample", inplace = True)

In [5]:
print("0 in features: \n", features.isnull().sum().sum())
print("NANs in features: \n", features.isna().sum().sum())
print("0 in labels: \n", labels.isnull().sum().sum())
print("NANs in labels: \n", labels.isna().sum().sum())

0 in features: 
 0
NANs in features: 
 0
0 in labels: 
 0
NANs in labels: 
 0


In [6]:
print("Are the sample arrays equal?", np.array_equal(np.array(features.index), np.array(labels.index)))

Are the sample arrays equal? True


In [7]:
# convert the pd dataframes into numpy vectors
print(features.shape)
print(labels.shape)
features = features.to_numpy(copy=True) 
labels = labels.to_numpy(copy=True)
print(features.shape)
print(labels.shape)


(481, 120)
(481, 1)
(481, 120)
(481, 1)


### train test split

In [8]:
# splitting into test and train
from sklearn.model_selection import train_test_split

X, y = features, labels.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33, shuffle = True)

In [9]:
# normalization of testst
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(X_train) # compute mean, std on train
X_test = transformer.transform(X_test)  # use same transformer for both, mimics that we do not know train data
X_train = transformer.transform(X_train)

In [10]:
X_test.sum()

-1616.8714329383492

### KNN model

In [86]:
# KNN Classifiers
from sklearn.neighbors import KNeighborsClassifier
# define parameters
n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [87]:
y_predicted = knn.predict(X_test)
knn_result = np.equal(y_predicted, y_test)
accuracy = (knn_result.sum() / len(knn_result)) * 100
print("Accuracy: " + str(round(accuracy,2)) + " %")

Accuracy: 80.41 %


In [88]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))
# how good we are to predicting classes, important to check both classes accuracy
# support: no of samples

              precision    recall  f1-score   support

           0       0.78      0.83      0.80        46
           1       0.83      0.78      0.81        51

    accuracy                           0.80        97
   macro avg       0.80      0.81      0.80        97
weighted avg       0.81      0.80      0.80        97



### Logitsic regression model

In [11]:
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression(penalty = "l1", solver = "liblinear", C = 2.) # bigger C lower pen -> 1 / C
logReg.fit(X_train, y_train)

LogisticRegression(C=2.0, penalty='l1', solver='liblinear')

In [12]:
y_predicted = logReg.predict(X_test)
logReg_result = np.equal(y_predicted, y_test)
accuracy = (logReg_result.sum() / len(logReg_result)) * 100
print("Accuracy: " + str(round(accuracy,2)) + " %")

Accuracy: 89.69 %


In [13]:
from sklearn.metrics import classification_report, f1_score
print(classification_report(y_test, y_predicted))
print(f1_score(y_test, y_predicted, average = "macro"))
# how good we are to predicting classes, important to check both classes accuracy
# support: no of samples

              precision    recall  f1-score   support

           0       0.85      0.96      0.90        46
           1       0.96      0.84      0.90        51

    accuracy                           0.90        97
   macro avg       0.90      0.90      0.90        97
weighted avg       0.90      0.90      0.90        97

0.8968962585034014


In [21]:
len(logReg.coef_[0].nonzero()[0])

12

### random Forrest

In [92]:
from sklearn.ensemble import RandomForestClassifier

ranFor = RandomForestClassifier(n_estimators = int(X_train.shape[1]), max_depth = 6) # hyper: choose 1
ranFor.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=120)

In [93]:
y_predicted = ranFor.predict(X_test)
ranFor_result = np.equal(y_predicted, y_test)
accuracy = (ranFor_result.sum() / len(ranFor_result)) * 100
print("Accuracy: " + str(round(accuracy,2)) + " %")

Accuracy: 82.47 %


In [94]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))
# how good we are to predicting classes, important to check both classes accuracy
# support: no of samples

              precision    recall  f1-score   support

           0       0.75      0.93      0.83        46
           1       0.93      0.73      0.81        51

    accuracy                           0.82        97
   macro avg       0.84      0.83      0.82        97
weighted avg       0.84      0.82      0.82        97



# ichorCNA

In [95]:
labels = pd.read_csv("Results/Machine_Learning/sample_map_ML.csv")
features  = pd.read_csv("Results/Machine_Learning/tumour_fractions_ML.csv")

# convert gender 
def binary(df, samples = True):
    
    cancer_list = ["A", "B", "C", "Lung", "Breast"]
    if samples == True:
        df["cancer"] = np.where(df["diagnosis"].isin(cancer_list), 1, 0)
        df.drop(columns = ["diagnosis"], inplace = True)
        df.set_index("sample", inplace = True)
        df.sort_index(axis = 0, inplace = True)
    else:
        df["gender_binary"] = np.where(df["gender"].isin(["male"]), 1, 0)
        df.drop(columns = ["gender"], inplace = True)
        df.set_index("sample", inplace = True)
        df.sort_index(axis = 0, inplace = True)
    
    return df

features = binary(features, samples = False)
labels = binary(labels, samples = True)
features.drop(axis = 0, labels = "EE87952", inplace = True)
labels.drop(axis = 0, labels = "EE88290", inplace = True)

In [96]:
np.array_equal(labels.index, features.index)

True

In [99]:
def splitter(labels, features):
    # convert to numpy
    features = features.to_numpy(copy=True) 
    labels = labels.to_numpy(copy=True)
    # split into train test
    X, y = features, labels.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33, shuffle = True)
    # normalize
    transformer = StandardScaler().fit(X_train) # compute mean, std on train
    X_test = transformer.transform(X_test)  # use same transformer for both, mimics that we do not know train data
    X_train = transformer.transform(X_train)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitter(labels, features)

In [101]:
logReg = LogisticRegression(penalty = "l1", solver = "liblinear", C = 1.) # bigger C lower pen -> 1 / C
logReg.fit(X_train, y_train)
y_predicted = logReg.predict(X_test)
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.62      0.98      0.76        46
           1       0.96      0.44      0.60        50

    accuracy                           0.70        96
   macro avg       0.79      0.71      0.68        96
weighted avg       0.79      0.70      0.68        96



In [118]:
logReg.coef_ # proposes dropping gender

array([[ 1.90666497,  0.38157269, -0.04271875]])

### LIQUORICE


In [4]:
labels = pd.read_csv("Results/Machine_Learning/sample_map_ML.csv")
features  = pd.read_csv("Results/Machine_Learning/cell_type_signatures_LIQU_ML.csv")
features = features.pivot(index = "sample", columns = "region_set", values = ["Dip_area", "Dip_depth"]) # first 13 dip_area

In [5]:
features = features.merge(labels, on = "samples", how = "inner")

C:\Users\paesc\AppData\Local\Temp\ipykernel_19184\1864539223.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  features = features.merge(labels, on = "samples", how = "inner")


KeyError: 'samples'

In [6]:
features


Dip_area                                       \
region_set A549_hg38 HPF_lung_fibroblast_cluster1737_hg38   
sample                                                      
EE87786         0.76                                -0.10   
EE87787        -2.16                                 1.22   
EE87788        -0.52                                -1.56   
EE87789        -2.78                                -1.38   
EE87790        -6.92                                -0.92   
...              ...                                  ...   
EE88321        -5.41                                -0.51   
EE88322        -0.75                                -0.43   
EE88323        -3.63                                 0.02   
EE88324        -0.32                                 1.91   
EE88325        -7.01                                -1.39   

                                                                  \
region_set HeLa_cluster1777_hg38 SAEC_hg38 colon_cluster507_hg38   
sample                                                             
EE87786                    -0.22     -0.78                  0.76   
EE87787                    -2.49     -3.51                  0.84   
EE87788                    -0.13     -0.22                  0.47   
EE87789                     0.19      0.14                  1.24   
EE87790                    -2.07     -2.13                 -0.05   
...                          ...       ...                   ...   
EE88321                    -2.30     -1.66                 -2.87   
EE88322                    -1.14     -1.56                 -1.06   
EE88323                    -1.29      1.37                 -0.69   
EE88324                    -0.15     -1.45                 -0.44   
EE88325                    -1.30      0.75                 -1.64   

                                                                      \
region_set hematopoietic_specific_liquorice_hg38 hepatocyte_all_hg38   
sample                                                                 
EE87786                                    -0.50               -0.55   
EE87787                                     1.25                1.52   
EE87788                                     0.94                0.09   
EE87789                                     2.05               -2.01   
EE87790                                     3.22               -5.01   
...                                          ...                 ...   
EE88321                                     3.07               -4.02   
EE88322                                     0.49               -0.49   
EE88323                                     1.11               -1.74   
EE88324                                    -0.41                0.30   
EE88325                                     4.07               -7.23   

                                                                               \
region_set mammary_epithel_cluster2438_hg38 melano_SOX_clusters1863_2205_hg38   
sample                                                                          
EE87786                                0.89                             -0.77   
EE87787                               -1.04                              1.00   
EE87788                               -0.45                              0.22   
EE87789                               -0.98                              0.22   
EE87790                               -0.43                             -0.33   
...                                     ...                               ...   
EE88321                                0.56                             -1.48   
EE88322                               -0.61                             -1.41   
EE88323                                1.32                              0.43   
EE88324                                1.20                              1.02   
EE88325                               -0.25                              1.94   

                                          ... Dip_depth     

In [ ]:
# drop row that is to much and index lalala